In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
# ============================================================
# ✅ FIX PATH FOR NOTEBOOK INSIDE /notebooks
# ============================================================
import sys, os
from pathlib import Path

# Xác định thư mục cha của notebooks/
project_root = Path(os.getcwd()).parent  # => C:\Users\User\RR_model
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))  # ⚠️ thêm project_root, KHÔNG thêm src

print("📁 Project root added:", project_root)

# Kiểm tra lại
import importlib.util
print("🔍 src module found:", importlib.util.find_spec("src"))

📁 Project root added: D:\Python_code\RR_model_1711
🔍 src module found: ModuleSpec(name='src', loader=<_frozen_importlib_external.SourceFileLoader object at 0x00000250FF391900>, origin='D:\\Python_code\\RR_model_1711\\src\\__init__.py', submodule_search_locations=['D:\\Python_code\\RR_model_1711\\src'])


In [3]:
# ===========================================================
# 📘 Roll Rate Demo v7 - Sử dụng dữ liệu thật từ Oracle hoặc SQL file
# ===========================================================

from pathlib import Path
import sys
import pandas as pd
import numpy as np

# --- 1️⃣ Setup đường dẫn và import các module ---
root = Path(".").resolve()
sys.path.append(str(root / "src"))

from src.data_loader import load_data
from src.config import CFG, OUT_ROOT
from src.rollrate.transition import compute_transition_from_pairs, compute_transition_by_mob
#from src.rollrate.forecast import forecast_report
from src.rollrate.export_excel import export_transition_excel
from src.rollrate.backtest import (
    compute_actual_matrix,
    rollforward_validation,
    plot_matrix_diff,
    plot_distribution_compare
)

print("✅ Import modules thành công.")
print("Cấu hình CFG:", CFG)


# --- 2️⃣ Load dữ liệu thực tế ---
sql_path = r"D:\Python_code\RR_Model_1711\sql\uns3.sql"

df = load_data(sql_path)
# df = load_data()
df['DISBURSAL_DATE'] = df['DISBURSAL_DATE'].apply(lambda x: x.replace(day=1))

#print(f"📦 Đã load {len(df):,} dòng dữ liệu từ {sql_path}")

# Kiểm tra các cột quan trọng
required_cols = [CFG["loan"], CFG["mob"], CFG["state"], CFG["ead"], CFG["cutoff"]]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"❌ Thiếu các cột cần thiết trong data: {missing}")

print("✅ Dữ liệu có đủ các cột cần thiết:", required_cols)

# Nếu chưa có PRODUCT_TYPE thì thêm mặc định
if "PRODUCT_TYPE" not in df.columns:
    df["PRODUCT_TYPE"] = "A"
    print("⚠️ Không có PRODUCT_TYPE trong data, gán mặc định = 'A'")

# Convert cutoff về datetime
df[CFG["cutoff"]] = pd.to_datetime(df[CFG["cutoff"]])





✅ Import modules thành công.
Cấu hình CFG: {'loan': 'AGREEMENT_ID', 'mob': 'MOB', 'state': 'STATE_MODEL', 'orig_date': 'DISBURSAL_DATE', 'ead': 'PRINCIPLE_OUTSTANDING', 'disb': 'DISBURSAL_AMOUNT', 'cutoff': 'CUTOFF_DATE'}
🔗 Loading data from Oracle...
=== SQL DEBUG ===
File: D:\Python_code\RR_Model_1711\sql\uns3.sql
First 200 chars:
 select * from TBL_RR_UNSECURED3
Params: {}


D:\Python_code\RR_model_1711\src\db.py:84: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(


🔗 Loading data from Oracle...


ValueError: Cần chỉ định tên SQL file hoặc câu SQL khi dùng Oracle.

In [4]:
df['DISBURSAL_DATE'] = df['DISBURSAL_DATE'].apply(lambda x: x.replace(day=1))

#print(f"📦 Đã load {len(df):,} dòng dữ liệu từ {sql_path}")

# Kiểm tra các cột quan trọng
required_cols = [CFG["loan"], CFG["mob"], CFG["state"], CFG["ead"], CFG["cutoff"]]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"❌ Thiếu các cột cần thiết trong data: {missing}")

print("✅ Dữ liệu có đủ các cột cần thiết:", required_cols)

# Nếu chưa có PRODUCT_TYPE thì thêm mặc định
if "PRODUCT_TYPE" not in df.columns:
    df["PRODUCT_TYPE"] = "A"
    print("⚠️ Không có PRODUCT_TYPE trong data, gán mặc định = 'A'")

# Convert cutoff về datetime
df[CFG["cutoff"]] = pd.to_datetime(df[CFG["cutoff"]])

✅ Dữ liệu có đủ các cột cần thiết: ['AGREEMENT_ID', 'MOB', 'STATE_MODEL', 'PRINCIPLE_OUTSTANDING', 'CUTOFF_DATE']


In [6]:
from src.rollrate.transition import compute_transition_by_mob
from src.rollrate.export_excel import export_transition_excel
from src.config import OUT_ROOT

print("\n=== Tính transition matrices theo Product × MOB × Score ===")
matrices_by_mob, parent_fallback = compute_transition_by_mob(df)

# Đếm real vs fallback
count_real = 0
count_fb = 0

for prod, mob_dict in matrices_by_mob.items():
    for mob, score_dict in mob_dict.items():
        for score, obj in score_dict.items():
            if obj["is_fallback"]:
                count_fb += 1
            else:
                count_real += 1

print(f"🔹 Ma trận REAL     : {count_real}")
print(f"🔸 Ma trận FALLBACK : {count_fb}")
print(f"🔢 Tổng             : {count_real + count_fb}")

# Xuất Excel
export_transition_excel(
    matrices_by_mob,
    parent_fallback,
    out_path=OUT_ROOT / "Transition_ByProduct_MOB_Score_2.xlsx"
)



=== Tính transition matrices theo Product × MOB × Score ===
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=500-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=550-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=600-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650+)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=OTHERS)
⚠️ Có 3 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'SOLDOUT']
⚙️ Built parent fallback for (product=SALPIL, score=A)
⚠️ Có

In [7]:


export_transition_excel(
    matrices_by_mob,
    parent_fallback,
    out_path=OUT_ROOT / "Transition_ByProduct_MOB_Score_2.xlsx"
)


✅ Xuất file transition thành công → D:\Python_code\RR_model_1711\outputs\Transition_ByProduct_MOB_Score_2.xlsx


In [8]:
# matrices_by_mob đã build từ module transition
# df = raw data chứa DISB_DATE, CUTOFF, MOB, PRINCIPLE_OUTSTANDING, STATE...

from src.rollrate.forecast import forecast_all_vintages


In [9]:
forecast_results = forecast_all_vintages(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=29,
    enable_macro=False
)


⚠️ Skip (TWLPIL, OTHERS, vintage=2025-03-01 00:00:00) due to error:
    'OTHERS'
⚠️ Skip (TWLPIL, OTHERS, vintage=2025-04-01 00:00:00) due to error:
    'OTHERS'
⚠️ Skip (TWLPIL, OTHERS, vintage=2025-05-01 00:00:00) due to error:
    'OTHERS'
⚠️ Skip (TWLPIL, OTHERS, vintage=2025-06-01 00:00:00) due to error:
    'OTHERS'
⚠️ Skip (TWLPIL, OTHERS, vintage=2025-07-01 00:00:00) due to error:
    'OTHERS'
⚠️ Skip (TWLPIL, OTHERS, vintage=2025-08-01 00:00:00) due to error:
    'OTHERS'
⚠️ Skip (TWLPIL, OTHERS, vintage=2025-09-01 00:00:00) due to error:
    'OTHERS'


In [10]:
from src.rollrate.lifecycle import (
    get_actual_all_vintages_amount,
    combine_all_lifecycle_amount,
    lifecycle_to_long_df_amount,
    build_full_lifecycle_amount,
    tag_forecast_rows_amount,
    add_del_metrics,
    aggregate_loss_to_product,
    aggregate_to_product,
    make_metric_pivot,
    export_lifecycle_all_products_one_file,
    aggregate_products_to_portfolio,
    extend_actual_info_with_portfolio
)


In [11]:
# # 1) Build lifecycle actual + forecast
# df_lifecycle = build_full_lifecycle_amount(df, matrices_by_mob, max_mob=29)

# # 2) Tag forecast
# df_lifecycle_tagged = tag_forecast_rows_amount(df_lifecycle, df)

# # 3) DEL30/60/90
# df_del = add_del_metrics(df_lifecycle_tagged, df)

# # 4) Aggregate lên Product
# df_del_prod = aggregate_to_product(df_del)

# # 5) Pivot lifecycle DEL30+
# df_del30_pivot = df_del_prod.pivot_table(
#     index="VINTAGE_DATE",
#     columns="MOB",
#     values="DEL30_PCT"
# ).fillna(0)

# df_del30_pivot.index.name = "Cohort"

# # 6) Actual max MOB (để kẻ đường đỏ)
# actual_info = (
#     df.groupby("DISBURSAL_DATE")["MOB"]
#     .max()
#     .to_dict()
# )

# # 7) Export Excel đầy đủ format
# export_heatmap(
#     df_pivot=df_del30_pivot,
#     df_del_prod=df_del_prod,
#     actual_info=actual_info,
#     filename="DEL30P_Lifecycle.xlsx"
# )

# print("🎉 Lifecycle DEL30+ đã xuất thành công!")


In [12]:
# # 1) Build lifecycle (actual + forecast, amount-based)
# df_lifecycle = build_full_lifecycle_amount(
#     df_raw=df,
#     matrices_by_mob=matrices_by_mob,
#     max_mob=29,
# )

# # 2) Tag forecast rows
# df_lifecycle_tagged = tag_forecast_rows_amount(
#     df_lifecycle=df_lifecycle,
#     df_raw=df,
# )

# # 3) Tính DEL30/60/90 amount + pct
# df_del = add_del_metrics(
#     df_lifecycle=df_lifecycle_tagged,
#     df_raw=df,
# )

# # 4) Aggregate Product_Score → Product
# df_del_prod = aggregate_to_product(df_del)

# # 5) Max actual MOB per (Product, Vintage)
# actual_info = (
#     df.groupby(["PRODUCT_TYPE", "DISBURSAL_DATE"])["MOB"]
#       .max()
#       .to_dict()
# )

# # 6) Export chung 1 file, nhiều sheet (mỗi Product × metric)
# export_lifecycle_all_products_one_file(
#     df_del_prod=df_del_prod,
#     actual_info=actual_info,
#     filename="Lifecycle_All_Products.xlsx",
# )


In [13]:
# # 6) Export chung 1 file, nhiều sheet (mỗi Product × metric)
# export_lifecycle_all_products_one_file(
#     df_del_prod=df_del_prod,
#     actual_info=actual_info,
#     filename="Lifecycle_UnSecured.xlsx",
# )


## New pipeline


In [14]:
# 1. Build lifecycle
df_lifecycle = build_full_lifecycle_amount(df, matrices_by_mob, max_mob=29)

# 2. Tag forecast rows
df_lifecycle = tag_forecast_rows_amount(df_lifecycle, df)

# 3. Add DEL metrics (level Product × Score × Vintage × MOB)
df_del = add_del_metrics(df_lifecycle, df)

# 4. Aggregate lên Product level
df_del_prod = aggregate_to_product(df_del)


⚠️ Skip (TWLPIL, OTHERS, vintage=2025-03-01 00:00:00) due to error:
    'OTHERS'
⚠️ Skip (TWLPIL, OTHERS, vintage=2025-04-01 00:00:00) due to error:
    'OTHERS'
⚠️ Skip (TWLPIL, OTHERS, vintage=2025-05-01 00:00:00) due to error:
    'OTHERS'
⚠️ Skip (TWLPIL, OTHERS, vintage=2025-06-01 00:00:00) due to error:
    'OTHERS'
⚠️ Skip (TWLPIL, OTHERS, vintage=2025-07-01 00:00:00) due to error:
    'OTHERS'
⚠️ Skip (TWLPIL, OTHERS, vintage=2025-08-01 00:00:00) due to error:
    'OTHERS'
⚠️ Skip (TWLPIL, OTHERS, vintage=2025-09-01 00:00:00) due to error:
    'OTHERS'


D:\Python_code\RR_model_1711\src\rollrate\lifecycle.py:371: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


In [1]:
# 5. Aggregate lên portfolio
df_portfolio = aggregate_products_to_portfolio(
    df_del_prod,
    portfolio_name="PORTFOLIO_ALL"   # hoặc tên tuỳ bạn
)

# 6. Gộp lại: product + portfolio
df_export = pd.concat(
    [df_del_prod, df_portfolio],
    ignore_index=True
)


NameError: name 'aggregate_products_to_portfolio' is not defined

In [16]:
# 5) Max actual MOB per (Product, Vintage)
actual_info = (
    df.groupby(["PRODUCT_TYPE", "DISBURSAL_DATE"])["MOB"]
      .max()
      .to_dict()
)

In [17]:
# 5. Aggregate lên portfolio
df_portfolio = aggregate_products_to_portfolio(
    df_del_prod,
    portfolio_name="PORTFOLIO_ALL"   # hoặc tên tuỳ bạn
)

# 6. Gộp lại: product + portfolio
df_export = pd.concat(
    [df_del_prod, df_portfolio],
    ignore_index=True
)


D:\Python_code\RR_model_1711\src\rollrate\lifecycle.py:437: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


In [18]:
actual_info_extended = extend_actual_info_with_portfolio(
    actual_info_prod=actual_info,
    portfolio_name="PORTFOLIO_ALL"
)

export_lifecycle_all_products_one_file(
    df_del_prod=df_export,
    actual_info=actual_info_extended,
    filename="Lifecycle_All_Products_And_Portfolio.xlsx"
)


✔ Export lifecycle multi-product thành công → Lifecycle_All_Products_And_Portfolio.xlsx


In [19]:
export_lifecycle_all_products_one_file(
    df_del_prod=df_export,
    actual_info=actual_info_extended,
    filename="Lifecycle_All_Products_And_Portfolio.xlsx"
)

✔ Export lifecycle multi-product thành công → Lifecycle_All_Products_And_Portfolio.xlsx


## sale plan

In [20]:
from src.rollrate.forecast_plan import forecast_sale_plan_by_mob


In [21]:
# Chuẩn hóa RISK_SCORE
def normalize_score(x):
    if pd.isna(x):
        return "ALL"
    x_str = str(x).strip().upper()
    if x_str in ["", "NA", "NAN", "NONE", "NULL"]:
        return "ALL"
    return x_str

sale_plan_df["RISK_SCORE"] = sale_plan_df["RISK_SCORE"].apply(normalize_score)


NameError: name 'sale_plan_df' is not defined

In [ ]:
sale_plan_df = pd.read_excel("C:/Users/User/RR_model/sale_plan.xlsx", sheet_name="SALE_PLAN")
sale_plan_df["VINTAGE_DATE"] = pd.to_datetime(sale_plan_df["VINTAGE_DATE"])

print("✔ Sale Plan loaded:")
display(sale_plan_df.head())


# ========================================
# 4) Forecast sale plan dùng matrices_by_mob thực tế
# ========================================
from src.config import BUCKETS_CANON, ABSORBING_BASE

states = list(dict.fromkeys(list(BUCKETS_CANON) + list(ABSORBING_BASE)))

# Xử lý SCORE an toàn
score_raw = row.get("RISK_SCORE", row.get("SCORE", None))

# Đồng nhất tất cả NA, NaN, None, 'NA', 'NAN', 'null', '' → "ALL"
if pd.isna(score_raw):
    score = "ALL"
else:
    score_str = str(score_raw).strip().upper()
    if score_str in ["", "NA", "NAN", "NONE", "NULL"]:
        score = "ALL"
    else:
        score = score_str


print("✔ Forecast completed! Sample:")
display(sale_plan_fc.head(10))

In [ ]:
import pandas as pd
import numpy as np

# Load sale plan
sale_plan_df = pd.read_excel(
    "C:/Users/User/RR_model/sale_plan.xlsx",
    sheet_name="SALE_PLAN"
)
sale_plan_df["VINTAGE_DATE"] = pd.to_datetime(sale_plan_df["VINTAGE_DATE"])

print("✔ Sale Plan loaded:")
display(sale_plan_df.head())

# ========================================
#  ⚠️ NORMALIZE SCORE – ĐÂY LÀ CHỖ SỬA QUAN TRỌNG
# ========================================

def normalize_score(x):
    # pd.NA, np.nan, None --> ALL
    if pd.isna(x):
        return "ALL"
    x_str = str(x).strip().upper()
    # string "NA", "NAN", "" -> ALL
    if x_str in ["", "NA", "NAN", "NONE", "NULL"]:
        return "ALL"
    return x_str

sale_plan_df["RISK_SCORE"] = sale_plan_df["RISK_SCORE"].apply(normalize_score)
print("✔ SCORE normalized:")
display(sale_plan_df.head())


In [ ]:
from src.rollrate.transition import compute_transition_by_mob, STATE_SPACE
from src.config import BUCKETS_CANON



sale_plan_fc = forecast_sale_plan_by_mob(
    sale_plan_df=sale_plan_df,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    mob_target=12,
    start_state=BUCKETS_CANON[0],
    states=STATE_SPACE,
)


In [ ]:
import pandas as pd
import numpy as np

from src.config import BUCKETS_CANON, ABSORBING_BASE
from src.rollrate.transition import compute_transition_by_mob, STATE_SPACE
from src.rollrate.forecast_plan import forecast_sale_plan_by_mob  # hàm bạn đặt trong forecast_plan.py


# ========================================
# 2) Load SALE PLAN + chuẩn hóa SCORE
# ========================================
sale_plan_df = pd.read_excel(
    "C:/Users/User/RR_model/sale_plan.xlsx",
    sheet_name="SALE_PLAN"
)
sale_plan_df["VINTAGE_DATE"] = pd.to_datetime(sale_plan_df["VINTAGE_DATE"])

print("✔ Sale Plan loaded:")
display(sale_plan_df.head())

# Chuẩn hóa RISK_SCORE
def normalize_score(x):
    if pd.isna(x):
        return "ALL"
    x_str = str(x).strip().upper()
    if x_str in ["", "NA", "NAN", "NONE", "NULL"]:
        return "ALL"
    return x_str

sale_plan_df["RISK_SCORE"] = sale_plan_df["RISK_SCORE"].apply(normalize_score)

print("✔ SCORE normalized:")
display(sale_plan_df["RISK_SCORE"].value_counts(dropna=False))

# ========================================
# 3) Forecast SALE PLAN
# ========================================
states = list(STATE_SPACE)  # dùng đúng state_space của engine

sale_plan_fc = forecast_sale_plan_by_mob(
    sale_plan_df=sale_plan_df,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    mob_target=12,
    start_state=BUCKETS_CANON[0],
    states=states,
)

print("✔ Forecast completed! Sample:")
display(sale_plan_fc.head(10))
